In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm.auto import tqdm

In [8]:
 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
class DCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(DCNN, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        
        #defining layers
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 40, kernel_size = (5,5), stride = (1,1), padding= (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (3,3),stride = (1,1))
        self.conv2 = nn.Conv2d(in_channels = 40, out_channels = 20, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.linear1 = nn.Linear(in_features = 20*24*24, out_features = num_classes)
        #defining activation function
        self.relu = nn.ReLU()
        
    def forward(self, X):
        X = self.relu(self.conv1(X))
        X = self.pool(X)
        X = self.relu(self.conv2(X))
        X = X.view(X.shape[0], -1)
        X = self.linear1(X)
        return X
    

#### Hyperparameters

In [29]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 10000
num_epochs = 100

#### Loading Data (Can also load from digit recognizer but have to do some preprocessing)

In [30]:
#Downloading Data Set of MNIST
train_dataset = datasets.MNIST(root="/kaggle/working/", train = True, transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root="/kaggle/working/", train = False, transform = transforms.ToTensor(), download = True)

In [31]:
#Creating Generator a.k.a Dataloader
train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#### Creating and Instance of model

In [32]:
model = DCNN(in_channels = in_channels, num_classes = num_classes)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
model = model.to(device)
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0
    for batch in train_dataloader:
        batch[0] = batch[0].to(device)
        batch[1] = batch[1].to(device)
        
        inference = model.forward(batch[0])
        
        loss = criterion(inference, batch[1])
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss
    if epoch%10 == 0:
        print(f"epoch:- {epoch.__str__().zfill(3)} loss :- {epoch_loss/len(train_dataloader)}")

  0%|          | 0/100 [00:00<?, ?it/s]

epoch:- 000 loss :- 1.8478405475616455
epoch:- 010 loss :- 0.10538534820079803
epoch:- 020 loss :- 0.0551096610724926
epoch:- 030 loss :- 0.04194357618689537
epoch:- 040 loss :- 0.03377971053123474
epoch:- 050 loss :- 0.028631336987018585
epoch:- 060 loss :- 0.02459508180618286
epoch:- 070 loss :- 0.021113146096467972
epoch:- 080 loss :- 0.01893586665391922
epoch:- 090 loss :- 0.015606189146637917


In [35]:
def evaluate(model, dataloader):
    model.eval()
    with torch.no_grad():
        num_correct = 0
        total_example = 0
        for batch in dataloader:
            batch[0] = batch[0].to(device)
            batch[1] = batch[1].to(device)

            inference = model.forward(batch[0])
            #output of max is max_value for each example with index location of the max in the last 10 neuron (y_pred) so no need of using softmax activation
            _, y_pred = torch.max(inference, dim = 1)
            num_correct += (y_pred == batch[1]).sum()
            total_example += inference.shape[0]
            
    print(f"Accuracy:- {num_correct/total_example}")
            

In [36]:
# Evaluation on Test Dataset
evaluate(model, test_dataloader)

Accuracy:- 0.9902999997138977


In [37]:
# Evaluation on Train Dataset
evaluate(model, train_dataloader)

Accuracy:- 0.9967666864395142
